In [4]:
### 6.4 Introduction
# RAG(Retrieval Augmented Generation: 검색 증강 생성)

from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()
vector = embedder.embed_documents([
    "hi", "how", "are", "you", "?"
])
print(len(vector), len(vector[0]))

5 1536


In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(
    file_path="./소나기 - 황순원.pdf"
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [25]:
result = vectorstore.similarity_search("소년과 소녀는 무었을 했는가?")

print(result, len(result))

[Document(page_content='수가 없었다.\n소년이 등을 돌려 댔다. 소녀가 순순히 업히었다. 걷어올린 소년의 잠방이까지 물이 올라\n왔다. 소녀는 ‘어머나’소리를 지르며 소년의 목을 끌어안았다.\n개울가에 다다르기 전에, 가을 하늘이 언제 그랬는가 싶게 구름 한 점 없이 쪽빛으로 개어\n있었다.\n그 뒤로 소녀의 모습은 뵈지 않았다. 매일같이 개울가로 달려와 봐도 뵈지 않았다.\n학교에서 쉬는 시간에 운동장을 살피기도 했다. 남 몰래 5학년 여자 반을 엿보기도 했다.\n그러나, 뵈지 않았다.', metadata={'source': './소나기 - 황순원.pdf'}), Document(page_content='수가 없었다.\n소년이 등을 돌려 댔다. 소녀가 순순히 업히었다. 걷어올린 소년의 잠방이까지 물이 올라\n왔다. 소녀는 ‘어머나’소리를 지르며 소년의 목을 끌어안았다.\n개울가에 다다르기 전에, 가을 하늘이 언제 그랬는가 싶게 구름 한 점 없이 쪽빛으로 개어\n있었다.\n그 뒤로 소녀의 모습은 뵈지 않았다. 매일같이 개울가로 달려와 봐도 뵈지 않았다.\n학교에서 쉬는 시간에 운동장을 살피기도 했다. 남 몰래 5학년 여자 반을 엿보기도 했다.\n그러나, 뵈지 않았다.', metadata={'source': './소나기 - 황순원.pdf'}), Document(page_content='수가 없었다.\n소년이 등을 돌려 댔다. 소녀가 순순히 업히었다. 걷어올린 소년의 잠방이까지 물이 올라\n왔다. 소녀는 ‘어머나’소리를 지르며 소년의 목을 끌어안았다.\n개울가에 다다르기 전에, 가을 하늘이 언제 그랬는가 싶게 구름 한 점 없이 쪽빛으로 개어\n있었다.\n그 뒤로 소녀의 모습은 뵈지 않았다. 매일같이 개울가로 달려와 봐도 뵈지 않았다.\n학교에서 쉬는 시간에 운동장을 살피기도 했다. 남 몰래 5학년 여자 반을 엿보기도 했다.\n그러나, 뵈지 않았다.', metadata={'source': './소나기 - 황순원.pdf'}), Document(pag

In [2]:
# 6.6 Legacy Chain Stuff

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(
    file_path="./소나기 - 황순원.pdf"
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstore.as_retriever(),
)

chain.run("이 소설의 등장 인물은 누구 있어?")

'주어진 문맥에서 등장하는 인물은 다음과 같습니다:\n\n- 소녀\n- 소년\n- 아버지'

In [ ]:
# 6.8 Stuff LCEL Chain

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")